In [1]:
import csv
from enum import Enum

import project_path
from model.data_accessor import get_data_file_abs_path

connectome_file = 'herm_full_edgelist.csv'

# key = conn_spec = (from, to)
# value = total weight
conn_spec_to_weight_chem = {}
conn_spec_to_weight_gap = {}

all_neurons = set()
with open(get_data_file_abs_path(connectome_file), newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  i = 0
  for row in reader:
    source = row['Source'].upper().strip()
    target = row['Target'].upper().strip()
    weight = row['Weight'].upper().strip()
    conn_type = row['Type'].upper().strip()
    conn_spec = (source, target)
    all_neurons.add(source)
    all_neurons.add(target)
    
    conn_spec_to_weight = None
    if conn_type == "CHEMICAL":
      conn_spec_to_weight = conn_spec_to_weight_chem
    elif conn_type == "ELECTRICAL":
      conn_spec_to_weight = conn_spec_to_weight_gap
    else:
      raise Exception("Invalid connection type: " + conn_type)
      
    if conn_spec in conn_spec_to_weight:
      raise Exception("Duplicate entry exists for %s. Previous value is %d, new value is %d" %\
                      (conn_type, conn_spec_to_weight[conn_spec], weight))
    conn_spec_to_weight[conn_spec] = weight


In [2]:
len(all_neurons)
# TODO: Why do we have 448 neurons? I noticed some neurons that have weird prefixes
# TODO: Identify which neurons were added, and if they are important.

448

In [7]:
from model.neuron_metadata import *
import numpy as np

neuron_metadata_collection = NeuronMetadataCollection.load_from_chem_json(get_data_file_abs_path('chem.json'))

def build_connectome_matrix(conn_spec_to_weight, neuron_metadata_collection):
  # TODO: should we use the new N not from chem.json, but from cook 2019?
  N = neuron_metadata_collection.get_size()
  mat = np.zeros((N,N))
  
  for conn_spec, weight in conn_spec_to_weight.items():
    source, target = conn_spec
    # TODO: What if not exists?
    source_id = neuron_metadata_collection.get_id_from_name(source)
    target_id = neuron_metadata_collection.get_id_from_name(target)
    should_skip = False
    if source_id < 0:
      should_skip = True
      print("Don't know " + source)
    if target_id < 0:
      should_skip = True
      print("Don't know " + target)
    # The existing Gg[i][j] means from neuron j to i.
    mat[target_id, source_id] = weight
  return mat
    
cook_Gg = build_connectome_matrix(conn_spec_to_weight_gap, neuron_metadata_collection)
cook_Gs = build_connectome_matrix(conn_spec_to_weight_chem, neuron_metadata_collection)


Don't know I1L
Don't know I2L
Don't know I1L
Don't know M3L
Don't know I1L
Don't know MCR
Don't know I1L
Don't know NSML
Don't know I1L
Don't know PM4VL
Don't know I1L
Don't know I1R
Don't know I2R
Don't know I1R
Don't know I3
Don't know I1R
Don't know MCR
Don't know I1R
Don't know NSMR
Don't know I1R
Don't know PM3VR
Don't know I1R
Don't know PM4VR
Don't know I1R
Don't know I2L
Don't know I1L
Don't know I2L
Don't know I4
Don't know I2L
Don't know M2L
Don't know I2R
Don't know I1R
Don't know I2R
Don't know M1
Don't know I2R
Don't know M2R
Don't know I2R
Don't know MCR
Don't know I2R
Don't know NSMR
Don't know I3
Don't know I1R
Don't know I3
Don't know M1
Don't know I3
Don't know MI
Don't know I3
Don't know PM4D
Don't know I3
Don't know PM5D
Don't know I4
Don't know I2L
Don't know I4
Don't know M2R
Don't know I4
Don't know M4
Don't know I4
Don't know PM5VL
Don't know I5
Don't know M2L
Don't know I5
Don't know M2R
Don't know I5
Don't know M5
Don't know I6
Don't know M4
Don't know M1
Don'